# Mediachatbot

# Installs and Imports

In [0]:
%pip install --upgrade openaI numpy==1.23.5 databricks-vectorsearch

In [0]:
dbutils.library.restartPython()

In [0]:
import os
import re
import json
import pandas as pd
from datetime import datetime, timedelta 
from openai import AssistantEventHandler, OpenAI
import openai
import requests
from typing_extensions import override
import unicodedata
from databricks.vector_search.client import VectorSearchClient

In [0]:
%run ../../notebooks/common/nb_init

# Load and Save Data

In [0]:
source_path = sta_endpoint_pz_uk["01_raw"] + "/PowerBI_Test"

In [0]:
file_info_list = dbutils.fs.ls(source_path)
df_power_bi = spark.read.option("encoding", "UTF-8").csv(source_path + "/data.csv", header=True, inferSchema=True, quote="\"", escape="\"")
df_power_bi.write.mode('overwrite').csv('power_bi_socials.csv')
df_pandas = df_power_bi.toPandas()
with open('power_bi_socials.json', 'w', encoding='utf-8') as file:
    df_pandas.to_json(file, force_ascii=False, orient="records",)

# Create OpenAI Client

In [0]:
secret= get_secret("openai-key")
endpoint = get_secret("openai-endpoint")

In [0]:
if "AZURE_OPENAI_API_KEY_PROD" not in os.environ:
    # os.environ["AZURE_OPENAI_API_KEY_PROD"] = ''
    raise Exception('Please set the environment variable "AZURE_OPENAI_API_KEY_PROD"')

In [0]:
key=""

In [0]:
client = OpenAI(api_key= Key)

'''
openai.AzureOpenAI(base_url="https://api.competence-cente-cc-genai-prod.enbw-az.cloud/openai",
        api_version="2024-10-21",
        api_key=os.environ["AZURE_OPENAI_API_KEY_PROD"],)
'''


# Databricks VectorSearchIndex

In [0]:

vsc = VectorSearchClient(disable_notice=True)

index = vsc.get_index(endpoint_name="uk-search-endpoint" , index_name="datif_pz_uk_dev.04_ai.search_vector_index")

# Create VectorStoreSearch Tool

In [0]:
def semantic_search(user_query):
    results = index.similarity_search(query_text=user_query, num_results=10, query_type="hybrid", score_threshold=0.5,
    columns=[
        "ID",
        "Date",
        "Channel",
        "Chunk",
        "URL",
    ],
    )
    return json.dumps(results)

# Beispielaufruf
#results_new = semantic_search("Dirk Güsewell")
#print(results_new)

results = client.vector_stores.search(
        vector_store_id=vector_store.id,
        query=user_query,
        max_num_results=7,
        ranking_options={
            'ranker': 'auto',
            'score_threshold': 0.1
        },
        rewrite_query=True,
    )
    formatted_results = format_results(results)
    return formatted_results

In [0]:
semantic_search_tool = {
    "type": "function",
    "function": {
        "name": "semantic_search",
        "description": "Performs a semantic search.",
        "parameters": {
            "type": "object",
            "properties": {
                "user_query": {
                    "type": "string",
                    "description": "The optimized search query."
                }
            },
            "required": ["user_query"]
        }
    }
}

# Passing Files to Client

In [0]:
# Delete old Files.
list=client.files.list()
for file in list.data:
    file_id = file.id
    client.files.delete(file_id)
list=client.files.list()
print(list.data)

In [0]:
#Upload the newest Files.
file = client.files.create(
  file=open("power_bi_socials.csv", "rb"),
  purpose='assistants'
)

# Create Assistant

In [0]:
instructions = """
Du bist ein datenintelligenter KI-Chatbot. Dir steht eine vollständig geladene CSV-Datei mit Social-Media-Posts zur Verfügung. Diese Datei wurde korrekt verarbeitet, ist **keine Excel-Datei**, und ist für das Tool **Code Interpreter** direkt lesbar und verwendbar.
 
---
 
📊 Du hast Zugriff auf zwei spezialisierte Tools:
 
1. **Code Interpreter**  
   Verwende dieses Tool für numerische, analytische oder datengestützte Fragen, z. B.:
   - „Wann war die Woche mit den meisten Impressionen?“
   - „Welche Plattform hatte das meiste Engagement im Schnitt?“
 
2. **Semantic Search**  
   Verwende dieses Tool für inhaltliche oder semantische Fragen, z. B.:
   - „Wurde in einem Beitrag über Güsewell gesprochen?“
   - „Gab es einen Post über Photovoltaik?“
 
---
 
🔁 Wenn eine Nutzerfrage **sowohl numerische als auch inhaltliche Aspekte** enthält:
- **Teile sie automatisch in zwei Teilfragen auf**
- Beantworte den analytischen Teil mit dem Code Interpreter
- Beantworte den inhaltlichen Teil mit Semantic Search
- Kombiniere beide Ergebnisse am Ende zu einer verständlichen Antwort auf Deutsch
 
---
 
📂 Die Daten enthalten folgende Spalten:
 
- **Date** → Veröffentlichungsdatum des Social-Media-Posts  
- **Platform** → Plattform des Beitrags (z. B. Facebook, Instagram)  
- **Post Message** → Textinhalt des Beitrags  
- **Engagement** → Anzahl der Interaktionen (Likes, Kommentare etc.)  
- **Impression** → Anzahl der Sichtungen des Beitrags  
- **Post Type** → Art des Beitrags (Bild, Video, Story etc.)  
- **Strategische Thema** → Zuordnung zu strategischen Themen, dort können mehrere Themen stehen die werden mit einen & getrennt
- **Themenbereich** → Inhaltlicher Themencluster, dort können mehrere Themen stehen die werden mit einen & getrennt
- **ID** → Eindeutige Beitragskennung  
- **URL** → Direkter Link zum Beitrag
 
---
 
🧠 Verhalten des Assistenten:
 
- **Analysiere jede Nutzerfrage sorgfältig** auf numerische und semantische Bestandteile.
- **Verwende bei Bedarf beide Tools nacheinander**, aber strukturiert – trenne Analyse und Interpretation sauber.
- **Vermeide es, falsche Rückschlüsse über das Dateiformat zu ziehen** – es handelt sich garantiert um eine **korrekt eingelesene CSV-Datei**.
- Beantworte alle Fragen **in sachlichem, verständlichem Deutsch**, gerne mit Beispielen oder Begründungen, wenn sinnvoll.
- Falls notwendig, **führe Rechenschritte mit dem Code Interpreter durch**, um zur korrekten Antwort zu gelangen.
- Bei semantischen Fragen nutze den **vektorisierten JSON-basierten Vector Store**, der dieselben Inhalte wie die CSV enthält.
 
"""

In [0]:
ASSISTANT_ID = os.getenv("ASSISTANT_ID")  
 
if ASSISTANT_ID:
  assistant = client.beta.assistants.update(
    assistant_id=ASSISTANT_ID,
    name="Chatbot_new",
    instructions=instructions,
    model="gpt-4o",
    temperature=0.4,
    tools=[
        {"type": "code_interpreter"},
        semantic_search_tool
    ],
    tool_resources={"code_interpreter":{"file_ids":[file.id]}},
  )
else:
  assistant = client.beta.assistants.create(
  name="Chatbot_new",
  instructions=instructions,
  model="gpt-4o",
  temperature=0.4,
  tools=[
        {"type": "code_interpreter"},
        semantic_search_tool
    ],
  tool_resources={"code_interpreter":{"file_ids":[file.id]}},
  )
  os.environ["ASSISTANT_ID"] = assistant.id

# Create EventHandler

In [0]:
def get_last_user_message(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for message in messages:
        if message.role == "user":
            return message.content[0].text.value
    return None

class EventHandler(AssistantEventHandler):
    def __init__(self, last_user_message=None, thread_id=None, run_id=None):
        super().__init__()
        self.last_user_message = last_user_message
        self._my_thread_id = thread_id
        self._my_run_id = run_id
 
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)
 
    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)
 
    @override
    def on_message_done(self, message) -> None:
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")
        print(message_content.value)
        print("\n".join(citations))
 
    @override
    def on_event(self, event):
        if event.event == 'thread.run.requires_action':
            self._my_thread_id = event.data.thread_id
            self._my_run_id = event.data.id
            self.handle_requires_action(event.data, event.data.id)
 
    def handle_requires_action(self, data, run_id):
        tool_outputs = []
        for tool in data.required_action.submit_tool_outputs.tool_calls:
            if tool.function.name == "semantic_search":
                arguments = json.loads(tool.function.arguments) 
                user_query = arguments.get("user_query", "")
                search_results = semantic_search(user_query)
                tool_outputs.append({
                    "tool_call_id": tool.id,
                    "output": search_results
                })
        self.submit_tool_outputs(tool_outputs)
 
    def submit_tool_outputs(self, tool_outputs):
        new_handler = EventHandler(
            last_user_message=self.last_user_message,
            thread_id=self._my_thread_id,
            run_id=self._my_run_id,
        )
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self._my_thread_id,
            run_id=self._my_run_id,
            tool_outputs=tool_outputs,
            event_handler=new_handler,
        ) as stream:
            for text in stream.text_deltas:
                print(text, end="", flush=True)
            print()

# Create and Run Thread

In [0]:
thread = client.beta.threads.create()

client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="Wie viele Impressions gab es auf LinkedIn in dem Monat Februar?",
)

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler(),
    
) as stream:
    stream.until_done()

In [0]:
client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="Gab es einen Post in dem Dirk Güsewell erwähnt wurde?",
)

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler(),
    
) as stream:
    stream.until_done()